In [12]:
import numpy as np
import os.path
from PIL import Image

In [13]:
DATA_DIR = '.'

ORIGINAL = os.path.join(DATA_DIR, 'Original.bmp')
FILTERED = os.path.join(DATA_DIR, 'RGB_CFA.bmp')

In [14]:
original = Image.open(ORIGINAL)
filtered = Image.open(FILTERED)
filtered_rgb = np.array(filtered.convert('RGB'), dtype='int64')
original_rgb = np.array(original.convert('RGB'), dtype='int64')

In [15]:
def normalize_color(value):
    value = min(value, 255)
    return max(value, 0)

# Compute unknown green values
# rb_m, g_m -- flattened matrix 5x5 for conresponding blocks
def calc_green(rb_m, g_m):
    gradient = {}
    gradient['N'] = abs(rb_m[12] - rb_m[2]) * 2 + abs(g_m[7] - g_m[17])
    gradient['E'] = abs(rb_m[12] - rb_m[14]) * 2 + abs(g_m[11] - g_m[13])
    gradient['W'] = abs(rb_m[12] - rb_m[10]) * 2 + abs(g_m[11] - g_m[13])
    gradient['S'] = abs(rb_m[12] - rb_m[22]) * 2 + abs(g_m[7] - g_m[17])
    
    min_gradient = min(gradient, key=gradient.get)
    g = 0
    if min_gradient == 'N':
        g = (g_m[7] * 3 + g_m[17] + rb_m[12] - rb_m[2]) / 4
    elif min_gradient == 'E':
        g = (g_m[13] * 3 + g_m[11] + rb_m[12] - rb_m[14]) / 4
    elif min_gradient == 'W':
        g = (g_m[11] * 3 + g_m[13] + rb_m[12] - rb_m[10]) / 4
    elif min_gradient == 'S':
        g = (g_m[17] * 3 + g_m[7] + rb_m[12] - rb_m[22]) / 4
    return normalize_color(g)

    
def hue_transit(L, V):
    if (L[0] < L[1] and L[1] < L[2]) or (L[0] > L[1] and L[1] > L[2]):
        return V[0] + (V[1] - V[0]) * (L[1] - L[0]) / (L[2] - L[0])
    else:
        return (V[0] + V[1]) / 2 + (L[1] - (L[0] + L[2]) / 2) / 2
    

# Compute red and blue for initially green pixels
# r_m, g_m, b_m -- flattened matrix 3x3 for conresponding blocks
def calc_blue_and_red(r_m, g_m, b_m, isRed):
    b = 0
    r = 0
    if isRed != Color.blue:
        b = hue_transit((g_m[3], g_m[4], g_m[5]), (b_m[3], b_m[5]))
        r = hue_transit((g_m[1], g_m[4], g_m[7]), (r_m[1], r_m[7]))
    else:
        r = hue_transit((g_m[3], g_m[4], g_m[5]), (r_m[3], r_m[5]))
        b = hue_transit((g_m[1], g_m[4], g_m[7]), (b_m[1], b_m[7]))
    return normalize_color(r), normalize_color(b)
    

# Compute red and blue for initially not green pixels
# rb_m, g_m -- flattened matrix 5x5 for conresponding blocks
def calc_red_or_blue(rb_m, g_m):
    gradient = {}
    gradient['NE'] = abs(rb_m[8] - rb_m[16]) + abs(rb_m[4] - rb_m[12]) + abs(rb_m[12] - rb_m[20]) + abs(g_m[8] - g_m[12]) + \
        abs(g_m[12] - g_m[16]) 
    gradient['NW'] = abs(rb_m[6] - rb_m[18]) + abs(rb_m[0] - rb_m[12]) + abs(rb_m[12] - rb_m[24]) + abs(g_m[6] - g_m[12]) + \
        abs(g_m[12] - g_m[18])
    rb = 0
    if gradient['NE'] < gradient['NW']:
        rb =  hue_transit((g_m[8], g_m[12], g_m[16]), (rb_m[8], rb_m[16]))
    else:
        rb = hue_transit((g_m[6], g_m[12], g_m[18]), (rb_m[6], rb_m[18]))
    return normalize_color(rb)

In [16]:
def to_list_9(m, y, x):
    return m[y-1:y+2, x-1:x+2].flatten()  

def to_list_25(m, y, x):
    return m[y-2:y+3, x-2:x+3].flatten() 

In [17]:
from enum import Enum


class Color(Enum):
    red, green, blue = range(3)

    
def get_cell_type(y, x):
    if y % 2 == 0:
        if x % 2 == 0:
            return Color.red
        else:
            return Color.green
    else:
        if x % 2 == 0:
            return Color.green
        else:
            return Color.blue

In [18]:
def ppg(img_rgb):
    img_r = img_rgb[:, :, 0]
    img_g = img_rgb[:, :, 1]
    img_b = img_rgb[:, :, 2]
    img_rb = img_r + img_b
    for y in range(2, img_rgb.shape[0] - 2):
        for x in range(2, img_rgb.shape[1] - 2):
            if get_cell_type(y, x) != Color.green:
                img_g[y][x] = calc_green(to_list_25(img_rb, y, x), to_list_25(img_g, y, x))
    for y in range(2, img_rgb.shape[0] - 2):
        for x in range(2, img_rgb.shape[1] - 2):
            if get_cell_type(y, x) == Color.green:
                img_r[y][x], img_b[y][x] = calc_blue_and_red(to_list_9(img_r, y, x), 
                                                             to_list_9(img_g, y, x), 
                                                             to_list_9(img_b, y, x),
                                                             get_cell_type(y + 1, x))
    for y in range(2, img_rgb.shape[0] - 2):
        for x in range(2, img_rgb.shape[1] - 2):
            if get_cell_type(y, x) != Color.green:
                if get_cell_type(y, x) != Color.red:
                    img_r[y][x] = calc_red_or_blue(to_list_25(img_rb, y, x), to_list_25(img_g, y, x))
                else:
                    img_b[y][x] = calc_red_or_blue(to_list_25(img_rb, y, x), to_list_25(img_g, y, x))
    for y in range(2, img_rgb.shape[0]):
        for x in range(2, img_rgb.shape[1]):
            img_rgb[y][x][0] = img_r[y][x]
            img_rgb[y][x][1] = img_g[y][x]
            img_rgb[y][x][2] = img_b[y][x]

In [19]:
ppg(filtered_rgb)

In [20]:
filtered_rgb = filtered_rgb.astype('uint8')
restored = Image.fromarray(filtered_rgb, mode='RGB')
restored.show()
restored.save('out.bmp')

In [21]:
import math

def calc_luminance(rgb):
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def PSNR(img1, img2):
    diff_sum = 0
    for y in range(img1.shape[0]):
        for x in range(img1.shape[1]):
            diff_sum += (calc_luminance(img1[y][x]) - calc_luminance(img2[y][x])) ** 2
    if diff_sum == 0 or img1.shape[0] == 0 or img1.shape[1] == 0:
        return None
    MSE = diff_sum / img1.shape[0] / img1.shape[1]
    return 20 * math.log(255, 10) - 10 * math.log(MSE, 10)

In [22]:
print(PSNR(original_rgb, filtered_rgb))

23.63182062759023
